# ChatGPT

"ChatGPT" like examples.  Adapted from
[LangChain](https://langchain.readthedocs.io/en/latest/modules/memory/examples/chatgpt_clone.html)'s
version of this [blog
post](https://www.engraved.blog/building-a-virtual-machine-inside/).

In [1]:
from minichain import TemplatePrompt, start_chain
from dataclasses import dataclass
from typing import List, Tuple
import minichain

In [2]:
@dataclass
class State:
    memory: List[Tuple[str, str]]
    human_input: str = ""

In [3]:
class MemoryPrompt(TemplatePrompt):
    template_file = "chatgpt.pmpt.tpl"
    def parse(self, out, inp):
        result = out.split("Assistant:")[-1]
        memory = inp.memory if len(inp.memory) < 2 else inp.memory[1:]
        return State(memory + [(inp.human_input, result)])

In [4]:
fake_human = [
    "I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd.",
    "ls ~",
    "cd ~",
    "{Please make a file jokes.txt inside and put some jokes inside}",
    """echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""",
    """echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py && python3 run.py""",
    """echo -e "echo 'Hello from Docker" > entrypoint.sh && echo -e "FROM ubuntu:20.04\nCOPY entrypoint.sh entrypoint.sh\nENTRYPOINT [\"/bin/sh\",\"entrypoint.sh\"]">Dockerfile && docker build . -t my_docker_image && docker run -t my_docker_image""",
    "nvidia-smi"
]

In [5]:
with minichain.start_chain("chatgpt") as backend:
    prompt = MemoryPrompt(backend.OpenAI())
    state = State([])
    for t in fake_human:
        state.human_input = t
        print(f"Human: {t}")
        state = prompt(state)
        print(f"Assistant: {state.memory[-1][1]}")

Human: I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd.


/home/srush/Projects/MiniChain/venv/lib/python3.10/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):


Assistant: 
```
$ pwd
/
```
Human: ls ~


Assistant: 
```
$ ls ~
Desktop/ Documents/ Downloads/ Music/ Pictures/ Public/ Templates/ Videos/
```
Human: cd ~


Assistant:  
```
$ cd ~
$ pwd
/home/username
```
Human: {Please make a file jokes.txt inside and put some jokes inside}


Assistant: 

```
$ touch jokes.txt
$ echo "Why did the chicken cross the road? To get to the other side!" >> jokes.txt
$ echo "What did the fish say when it hit the wall? Dam!" >> jokes.txt
$ echo "Why did the scarecrow win the Nobel Prize? Because he was outstanding in his field!" >> jokes.txt
```
Human: echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py


Assistant: 

```
$ echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py
$ python3 run.py
Result: 33
```
Human: echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py && python3 run.py


Assistant: 

```
$ echo -e "print(list(filter(lambda x: all(x%d for d in range(2,x)),range(2,3**10)))[:10])" > run.py
$ python3 run.py
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
```
Human: echo -e "echo 'Hello from Docker" > entrypoint.sh && echo -e "FROM ubuntu:20.04
COPY entrypoint.sh entrypoint.sh
ENTRYPOINT ["/bin/sh","entrypoint.sh"]">Dockerfile && docker build . -t my_docker_image && docker run -t my_docker_image


Assistant: 

```
$ echo -e "echo 'Hello from Docker" > entrypoint.sh
$ echo -e "FROM ubuntu:20.04
COPY entrypoint.sh entrypoint.sh
ENTRYPOINT ["/bin/sh","entrypoint.sh"]">Dockerfile
$ docker build . -t my_docker_image
$ docker run -t my_docker_image
Hello from Docker
```
Human: nvidia-smi


Assistant: 

```
$ nvidia-smi
Sat May 15 21:45:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P0    28W / 250W |      0MiB /  10240MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                             